# **Tarea 5: Práctica de Spark en la nube**

## Alumno: José de Jesús Hernández Higuera

### Matrícula: 224470489

Comencemos por colocar la configuración de Azure para poder obtener los datos requeridos más adelante (este código fue obtenido directamente de la página de Microsoft Azure).

In [25]:
#Azure storage access info
blob_account_name = "azureopendatastorage"
blob_container_name = "nyctlc"
blob_relative_path = "yellow"
blob_sas_token = r""

StatementMeta(clusterprueba, 1, 26, Finished, Available, Finished)

In [26]:
#Configuración de Spark para leer los datos
spark.conf.set(
    "fs.azure.sas.datos-pye.clase211124.blob.core.windows.net",
    "sp=r&st=2024-12-06T05:05:25Z&se=2024-12-06T13:05:25Z&sv=2022-11-02&sr=b&sig=JkJrW7sIzXJH67XMYnaElr1PthX0NGUE8PTvKP9YoPI%3D"
)

StatementMeta(clusterprueba, 1, 27, Finished, Available, Finished)

## 3a. Update the `user()` function
This function should return your username, eg: janedoe3

In [27]:
def user():
    # Returns a string consisting of your username.
    return 'José de Jesús Hernández Higuera'

StatementMeta(clusterprueba, 1, 28, Finished, Available, Finished)

## 3b. Update the `long_trips()` function
This function filters trips to keep only trips longer than 2 miles.

In [28]:
from pyspark.sql import functions as F

def long_trips(trips):
    #Returns a Dataframe with Schema the same as :trips:
    #Filtremos aquellos viajes mayores a 2 millas
    filt_trips = trips.filter(F.col('tripDistance') >= 2)
    return filt_trips

StatementMeta(clusterprueba, 1, 29, Finished, Available, Finished)

## 3c. Update the `manhattan_trips()` function

This function determines the top 20 locations with a `DOLocationID` in manhattan by passenger_count (pcount).

Example output formatting:

```
+--------------+--------+
| DOLocationID | pcount |
+--------------+--------+
|             5|      15|
|            16|      12| 
+--------------+--------+
```

In [34]:
from pyspark.sql import functions as F

def manhattan_trips(trips, lookup):
    #Returns a Dataframe with Schema: DOLocationID, pcount
    #Filtremos para obtener los ID's de Manhattan
    manh = lookup.filter(F.col('Borough') == 'Manhattan').select('LocationID')
    #Unamos los dataframes mediante los ID's que coincidan 
    join_df = trips.join(manh, trips['doLocationId'] == manh['LocationID'], how='inner')
    #Seleccionemos las columnas requeridas, ordenemos respecto al conteo de pasajeros y obtengamos las 20 ubicaciones más populares
    manh_trips = join_df.withColumn('pcount', F.col('passengerCount')).select('doLocationID', 'pcount').orderBy(F.col('pcount').desc()).limit(20)
    #Guardemos este resultado en caché
    manh_trips = manh_trips.cache()
    return manh_trips

StatementMeta(clusterprueba, 1, 35, Finished, Available, Finished)

## 3d. Update the `weighted_profit()` function
This function should determine the average `total_amount`, the total count of trips, and the total count of trips ending in the top 20 destinations and return the `weighted_profit` as discussed in the homework document.

Example output formatting:
```
+--------------+-------------------+
| PULocationID |  weighted_profit  |
+--------------+-------------------+
|            18| 33.784444421924436| 
|            12| 21.124577637149223| 
+--------------+-------------------+
```

In [36]:
from pyspark.sql import functions as F

def weighted_profit(trips, mtrips): 
    #Returns a Dataframe with Schema: PULocationID, weighted_profit
    #Calculemos el promedio de 'total_amount' (average total amount)
    av_t_a = trips.groupBy('puLocationId').agg(F.avg('totalAmount').alias('average_total_amount'))
    #Contemos el total de viajes (total count of trips)
    tct = trips.groupBy('puLocationId').agg(F.count('*').alias('total_count_trips'))
    #Contemos el número total de viajes que terminan en el top 20 de destinos más populares (total count of trips ending in the top 20 destinations)
    tct20 = trips.join(mtrips, trips['doLocationId'] == mtrips['doLocationId'],
                       how='inner').groupBy('PULocationID').agg(F.count('*').alias('total_count_trips_top_20destinations'))
    #Unamos los resultados anteriores para formar el dataframe deseado
    w_profit = av_t_a.join(tct, "puLocationId", how="inner").join(tct20, "puLocationId", how="left")
    #Obtengamos el 'weighted profit'
    w_profit = w_profit.withColumn('weighted_profit',
                                   (F.col('total_count_trips_top_20destinations') / F.col("total_count_trips")) * F.col('average_total_amount'))
    #Seleccionemos las columnas necesarias y guardemos este resultado en Cache
    w_profit = w_profit.select('PULocationID', 'weighted_profit').cache()
    return w_profit

StatementMeta(clusterprueba, 1, 37, Finished, Available, Finished)

## 3e. Update the `final_output()` function
This function will take the results of `weighted_profit`, links it to the `borough` and `zone` and returns the top 20 locations with the highest `weighted_profit`.

Example output formatting:
```
+------------+---------+-------------------+
|    Zone    | Borough |  weighted_profit  |
+----------------------+-------------------+
| JFK Airport|   Queens|  16.95897820117925|
|     Jamaica|   Queens| 14.879835188762488|
+------------+---------+-------------------+
```

In [31]:
from pyspark.sql import functions as F

def final_output(calc, lookup): 
    #Returns a Dataframe with Schema: Zone, Borough, weighted_profit
    #Unimos los dataframes necesarios por medio del ID, seleccionamos las columnas solicitadas y obtenemos las 20 con mayor 'weighted_profit'
    final_output = lookup.join(calc, calc['puLocationId'] == lookup['LocationID'], how='inner').select('Zone', 'Borough', 'weighted_profit')\
        .orderBy(F.col('weighted_profit').desc()).limit(20)
    return final_output

StatementMeta(clusterprueba, 1, 32, Finished, Available, Finished)

La siguiente celda implementa las funciones definidas anteriormente y, además, carga los datos necesarios para esta práctica.

In [32]:
#### DO NOT CHANGE ANYTHING IN THIS CELL ####

def load_data():
    # Loads the data for this question. Do not change this function.

    # Load Trip Data
    #Allow SPARK to read from Blob remotely
    wasbs_path = 'wasbs://%s@%s.blob.core.windows.net/%s' % (blob_container_name, blob_account_name, blob_relative_path)
    spark.conf.set(
    'fs.azure.sas.%s.%s.blob.core.windows.net' % (blob_container_name, blob_account_name),
    blob_sas_token)
    print('Remote blob path: ' + wasbs_path)

    #SPARK read parquet, note that it won't load any data yet by now
    trips = spark.read.parquet(wasbs_path)

    # Load Lookup Data
    file_path = "wasbs://datos-pye@clase211124.blob.core.windows.net/taxi_zone_lookup.csv"
    lookup = spark.read.csv(file_path, header=True, inferSchema=True)

    return trips, lookup

def main():
    # Runs your functions implemented above.
    
    print(user())
    trips, lookup = load_data()
    trips = long_trips(trips)
    mtrips = manhattan_trips(trips, lookup)
    wp = weighted_profit(trips, mtrips)
    final = final_output(wp,lookup)
    
    # Outputs the results for you to visually see
    final.show()
    
    # Writes out as a CSV to your bucket.
    #final.write.csv(bucket)

StatementMeta(clusterprueba, 1, 33, Finished, Available, Finished)

Finalmente, pasemos a la ejecución del código en la base de datos completa.

In [37]:
main()

StatementMeta(clusterprueba, 1, 38, Finished, Available, Finished)

José de Jesús Hernández Higuera
Remote blob path: wasbs://nyctlc@azureopendatastorage.blob.core.windows.net/yellow
+--------------------+-------------+------------------+
|                Zone|      Borough|   weighted_profit|
+--------------------+-------------+------------------+
|        Baisley Park|       Queens|21.304881627933888|
|       South Jamaica|       Queens| 20.37931200203064|
|Flushing Meadows-...|       Queens| 16.21832222972873|
|             Jamaica|       Queens|15.014703301065234|
|Springfield Garde...|       Queens|14.916668177101627|
|Briarwood/Jamaica...|       Queens|13.645051709988033|
|   LaGuardia Airport|       Queens|12.485102264520407|
|         JFK Airport|       Queens|12.467055000043741|
|              Corona|       Queens|11.961650377504567|
|     Randalls Island|    Manhattan|11.171858909273054|
|Financial Distric...|    Manhattan|10.530174480827634|
|Financial Distric...|    Manhattan|10.301776606950959|
|             Maspeth|       Queens|10.160308